In [1]:
# Import all required libraries
import pandas as pd
from pandas import *
import numpy
from datetime import datetime

# Initialize values

env = None
sample_size = None
# Set env, if env = test, will only be run locally and display the result
env = "prod"
env = "test"

nb_models = 10
nb_min_value = 30

# Number of value on which to train, if null, train on all value
sample_size = None
test_size = 1000

In [2]:
# Read training data + test data
df_data_input = pandas.read_json("../input/train.json")
df_test_input = pandas.read_json("../input/test.json")
df_sample_input = pandas.read_csv("../input/sample_submission.csv")

print("df_data contais {0} elements".format(df_data_input.count()))
# Display basic information
display(df_data_input.head(3))
display(df_test_input.head(3))
display(df_sample_input.head(2))

df_data contais cuisine        39774
id             39774
ingredients    39774
dtype: int64 elements


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."


,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."


,id,cuisine
0,35203,italian
1,17600,italian


In [3]:
if env == "test":
    if sample_size is not None and sample_size > 0:
        df_data = df_data_input.sample(sample_size)
    else:
        df_data = df_data_input
    df_test = df_data_input.sample(test_size)
    
    # Removing all df_test from df_data to ensure not train with test data
    df_common = df_data.merge(df_test,on=['id'])
    #display(df_common)
    df_data = df_data[(~df_data.id.isin(df_common.id))]
else:
    # set that to some default value
    df_test = df_test_input
    df_test['cuisine'] = "todo"
    if sample_size is not None and sample_size > 0:
        df_data = df_data_input.sample(sample_size)
    else:
        df_data = df_data_input
        
print("df_data contais {0} elements".format(df_data.count()))
# Display basic information
display(df_data.head(3))
display(df_test.head(3))

df_data contais cuisine        38774
id             38774
ingredients    38774
dtype: int64 elements


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."


,cuisine,id,ingredients
37231,brazilian,13802,"[water, margarine, eggs, active dry yeast, bre..."
30430,italian,14311,"[lasagna noodles, sauce, parmigiano reggiano c..."
19399,british,44867,"[eggs, all-purpose flour, prepared horseradish..."


In [4]:
# Multiply the training data set for food where there is not that much data
df_count = df_data.groupby('cuisine')['cuisine']
max_recipe_count_per_cuisine = df_count.count().max()
# Loop over all cuisine, if while < minimal, add this dataframe, then add a sample of those, to get exactly the same number

print("Max value per cuisine = {0}".format(max_recipe_count_per_cuisine))
df_data_a = df_data
for cuisine in df_data_a.cuisine.unique():
    # nb for cuisine
    df_cuisine = df_data_a.loc[df_data_a['cuisine'] == cuisine]
    recipe_count = df_cuisine.shape[0]
    nb_recipe_to_add = max_recipe_count_per_cuisine - recipe_count
    print("Got {0} value for {1}, need to add {2}".format(recipe_count, cuisine, nb_recipe_to_add))
    tmp_df = None
    if nb_recipe_to_add != 0:
        while nb_recipe_to_add != 0:
            if nb_recipe_to_add >= recipe_count:
                # Add the full dataframe
                if tmp_df is None:
                    tmp_df = df_cuisine
                else:
                    tmp_df = tmp_df.append(df_cuisine, ignore_index=True)
                nb_recipe_to_add -= recipe_count
            else:
                # Only add a sample of it
                if tmp_df is None:
                    tmp_df = df_cuisine
                else:
                    tmp_df = tmp_df.append(df_cuisine.sample(nb_recipe_to_add), ignore_index=True)
                nb_recipe_to_add = 0
            # Add tmp df to df_data
        df_data_a = df_data_a.append(tmp_df, ignore_index=True)
        print("Append a dataframe of {0} values".format(tmp_df.shape[0]))



Max value per cuisine = 7661
Got 1149 value for greek, need to add 6512
Append a dataframe of 6512 values
Got 4200 value for southern_us, need to add 3461
Append a dataframe of 4200 values
Got 738 value for filipino, need to add 6923
Append a dataframe of 6923 values
Got 2922 value for indian, need to add 4739
Append a dataframe of 4739 values
Got 513 value for jamaican, need to add 7148
Append a dataframe of 7148 values
Got 964 value for spanish, need to add 6697
Append a dataframe of 6697 values
Got 7661 value for italian, need to add 0
Got 6293 value for mexican, need to add 1368
Append a dataframe of 6293 values
Got 2590 value for chinese, need to add 5071
Append a dataframe of 5071 values
Got 787 value for british, need to add 6874
Append a dataframe of 6874 values
Got 1494 value for thai, need to add 6167
Append a dataframe of 6167 values
Got 804 value for vietnamese, need to add 6857
Append a dataframe of 6857 values
Got 1508 value for cajun_creole, need to add 6153
Append a dat

In [ ]:
# Extract columns values of ingredients to multiple columns with boolean
import re
def preprocess_dataframe(df1, df2):
    count1 = len(df1)
    count2 = len(df2)
    
    
    
    print("count1: {0}".format(count1))
    print("count2: {0}".format(count2))
    
    total_df = df1.append(df2, ignore_index=True)
    
    d_list = []
    
    output = []
    all_ingredients = {}
    for index, row in total_df.iterrows():
        for value_field in row['ingredients']:
            # Transform the value to remove number and percentage
            values = re.split('\s+', value_field)
            for value in values:
                # build a dictionnary of all values
                if value not in output:
                    output.append(value)
                    all_ingredients[value] = 1
                else:
                    all_ingredients[value] += 1
                d_list.append({'id':row['id'], 
                               'value':value})

    total_df = total_df.append(d_list, ignore_index=True)
    total_df = total_df.groupby('id')['value'].value_counts()
    total_df = total_df.unstack(level=-1).fillna(0)
        
    
    # Then, we need to merge df_1 and df_2 with their id
    df1 = df1.merge(total_df, left_on='id', right_on='id', how='inner')
    df2 = df2.merge(total_df, left_on='id', right_on='id', how='inner')

    # We do not need the ingredients column now, so, we can remove it
    df1 = df1.drop(columns=['ingredients'])
    df2 = df2.drop(columns=['ingredients'])

    return df1, df2, all_ingredients

df_data_0, df_test_0, all_ingredients = preprocess_dataframe(df_data_a, df_test)



def category_to_number(df):
    df["code_cuisine"] = df.cuisine.astype("category").cat.codes
    mapping = df[["cuisine", "code_cuisine"]]
    return df, mapping.drop_duplicates()

#df_data_1, code_mapping = category_to_number(df_data_1)
#df_test_1, empty = category_to_number(df_test_1)


display(df_data_a.head(5))
display(df_data_0.head(5))

count1: 158884
count2: 1000


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
# Check dictionnary and create df_data_1 and df_test_1 from it
output = {}
for key, value in all_ingredients.items():
    if value not in output:
        output[value] = [key]
    else:
        output[value].append(key)
#print(output)

# Shuffle the data as well
#df = df.sample(frac=1).reset_index(drop=True)
df_data_1 = df_data_0.sample(frac=1).reset_index(drop=True)
df_test_1 = df_test_0

#print(output)
to_remove = []
for key, values in output.items():
    for value in values:
        if key < nb_min_value:
            #print("Removing {0}".format(value))
            to_remove.append(value)
df_data_1 = df_data_1.drop(columns=to_remove)
df_test_1 = df_test_1.drop(columns=to_remove)

#TODO multiply the training data by cuisine type as well

display(df_data_1.head(5))

In [ ]:
# Generate our training/validation datasets
from sklearn import model_selection

# Name of the result column
result_cols = ['cuisine']
result_excl_cols = ['cuisine_']

input_cols = [
    ''
]
input_excl_cols = ['ingredients', 'cuisine']
# Removing input_cols = ['store', 'item',
# dom, cw, 

# Train on everything

# Get the final values
def get_values(df, cols=[], excl_cols = []):
    columns = df.columns.values
    # Remove all columns that are not inside the list
    cols_to_drop = []
    for column in columns:
        find = False
        ignore = False
        for excl_col in excl_cols:
            if column.startswith(excl_col):
                ignore = True
        if ignore is False:
            for col in cols:
                if column.startswith(col):
                    find = True
        if not find:
            cols_to_drop.append(column)
    print("dropping columns")
    df = df.drop(columns=cols_to_drop)
    print("end dropping columns")
    new_order = sorted(df.columns.values)
    # Same order for both training and testing set
    df = df[new_order]
    return df.values

X_train = get_values(df_data_1, input_cols, input_excl_cols)
X_test = get_values(df_test_1, input_cols, input_excl_cols)

Y_train = get_values(df_data_1, result_cols, result_excl_cols).ravel()

# In test env, we calculate it for the test only
if env == "test":
    Y_test = get_values(df_test_1, result_cols, result_excl_cols).ravel()

In [ ]:
# Normalize the data


X_all = [x + y for x, y in zip(X_train, X_test)]

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler() 

# Don't cheat - fit only on training data
# Def adding x_train + X_test + X_validation to fit all of them
scaler.fit(X_all)  

X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 

In [ ]:
# Custom function to calculate the SMAPE
def get_score(Y_validation, Y_validation_predict):
    nb_success = 0
    for i in range(0, len(Y_validation)):
        if Y_validation[i] == Y_validation_predict[i]:
            nb_success += 1
    return nb_success / len(Y_validation) * 100

In [ ]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

import lightgbm as lgbm

models = []

#models.append(('LogisticRegression', LogisticRegression()))
#models.append(('KNeighborsClassifier', KNeighborsClassifier()))
#models.append(('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()))
#models.append(('GaussianNB', GaussianNB()))
#models.append(('SVC', SVC()))

for i in range(5, 5 + nb_models):
    #models.append(('MLPClassifier_adamrelu_{0}'.format(i), MLPClassifier(hidden_layer_sizes=(i,), 
    #                                                            activation='relu', 
    #                                                            solver='adam',
    #                                                            alpha=0.001, 
    #                                                            batch_size='auto',
    #learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    #random_state=i, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    #early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)))
    
    # Try multiple solutions
    hidden_layers = (50, )
    
    models.append(('MLPClassifier_adamrelu_earlystopping_{0}'.format(i), MLPClassifier(hidden_layer_sizes=hidden_layers, 
                                                                activation='logistic', 
                                                                solver='adam',
                                                                alpha=0.001, 
                                                                batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=i, tol=0.00001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)))
#models = []
#models.append(('lgbm', lgbm.sklearn.LGBMRegressor()))
# High value until first model get solved
best_model = "UNKNOWN"

res = []
# Testing all models, one by one
for name, model in models:
    print("Executing for model {0}".format(name))
    time_start = datetime.now()

    # Training the model
    model.fit(X_train, Y_train)
    
    print("Finish fit for {0}".format(name))

    Y_test_result = model.predict(X_test)
    res.append(Y_test_result)
    if env == "test":
        # We can calculate the avg error
        score = get_score(Y_test, Y_test_result)
        print("Model {0} got score of {1}, time: {2}".format(name, score, datetime.now() - time_start))
    else:
        # Let's write an output file, with the name of the model
        print("Writing output file {0}.csv for model {0}".format(name))
        
        df_test['cuisine'] = Y_test_result
        result_df = df_test[['id', 'cuisine']]
        result_df['cuisine'] = Y_test_result
        
        result_df.to_csv("{0}.csv".format(name), index=False)

In [ ]:
# For all result in res, if test, display the result, if not, write it to a file
final_res = []
nb_variable = len(res[0])
for variable in range(0, nb_variable):
    final_res.append(0.0)
    dict_cuisine = {}
    for i in range(0, len(res)):
        cuisine_found = res[i][variable]
        if cuisine_found in dict_cuisine:
            dict_cuisine[cuisine_found] += 1
        else:
            dict_cuisine[cuisine_found] = 1
    # Now, we need to find the most common one for all the values inside dict_cuisine
    current_value = 0
    current_cuisine = ""
    for cuisine in dict_cuisine:
        if dict_cuisine[cuisine] > current_value:
            current_cuisine = cuisine
            current_value = dict_cuisine[cuisine]
    
    final_res[variable] = current_cuisine

if env == "test":
    # We can calculate the avg error
    score = get_score(Y_test, final_res)
    print("avg model got score of {0}".format(score))
else:
    print("Writing output file merged.csv".format(name))

    df_test['cuisine'] = final_res
    result_df = df_test[['id', 'cuisine']]
    result_df['cuisine'] = final_res

    result_df.to_csv("merged.csv".format(name), index=False)